# 1. Importações

In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# 2. Leitura dos Dados

In [3]:
path = '/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/02. Arquivos/01. Dados/01. Dados para treino e teste/'
x_train = pd.read_csv(path + 'x_train.csv', index_col=False).drop('Unnamed: 0', axis=1)
x_test = pd.read_csv(path + 'x_test.csv', index_col=False).drop('Unnamed: 0', axis=1)
y_train = pd.read_csv(path + 'y_train.csv', index_col=False).drop('Unnamed: 0', axis=1)
y_test = pd.read_csv(path + 'y_test.csv', index_col=False).drop('Unnamed: 0', axis=1)

len(x_train), len(x_test), len(y_train), len(y_test)

(1106, 277, 1106, 277)

In [ ]:
x_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,0.336497,1.050568,-0.262547,0.028832,0.546745,0.103151,0.486898,0.221783,0.186779,-0.245445,0.649411,-0.262438,-0.017334,-0.003588,0.738577,-0.199617,-1.170416,0.168177,-0.721908,-0.575430,-0.041997,-0.037947,0.136671,-0.226719,-0.301812,0.547246,0.038292,0.297607,0.141801,0.190675,0.131600,0.065067,0.318491,0.155697,0.553171,-0.569110,0.438688,-0.418162,0.760687,0.196068,...,-0.585482,0.897799,0.312153,0.314197,0.375316,0.898591,-0.369271,0.267432,0.496364,-0.004315,-0.354653,-0.201504,1.031489,0.711821,-0.423715,0.435831,-0.616887,-0.079949,0.619846,0.272555,-0.353742,0.050269,0.056275,0.136378,0.128600,-0.020907,0.197173,-0.868580,-0.303744,0.457383,1.024805,-0.158959,0.199840,-0.435081,-0.444681,0.355497,0.209040,0.086414,0.187023,0.300691
1,1.265086,-0.078534,0.459882,0.591173,-0.384565,0.285756,-0.830370,1.859362,-1.045275,-0.321382,-1.061868,-0.004171,-0.412047,0.355199,0.353647,1.172089,-1.451156,-0.822381,-0.315141,0.220325,0.353405,-0.196840,0.139096,0.565915,-0.505101,-0.863965,-0.813413,-0.435851,-0.314876,1.721650,1.215067,-0.577375,0.064546,0.222700,-0.937205,0.668935,-0.587095,-0.006672,0.038139,0.431962,...,-0.093423,0.939042,0.258736,-0.268880,-0.398409,-0.801387,-1.409500,-0.522308,-0.044514,1.142100,0.443368,-0.337162,0.474103,0.287923,0.059692,1.656087,-0.172595,-0.296734,-0.032575,1.017213,0.072792,0.069213,0.394325,-0.280747,-0.648188,0.035106,0.677617,0.168221,-1.171392,-1.272269,1.393987,1.195631,0.239411,-0.906426,0.108893,0.545174,-0.338858,0.669485,0.157073,-0.186110
2,-0.265787,-0.020645,0.471996,-0.419554,1.062484,-0.652388,0.675918,1.295728,-0.185714,0.144719,-1.080305,0.505804,0.513946,-0.742673,0.950062,0.784561,-1.157126,-1.413858,0.194380,0.792449,-0.604863,0.079412,0.210165,0.296771,-0.129580,-0.592935,0.290979,-0.651587,1.008977,1.786370,0.859940,-1.039212,0.521774,-0.715076,-0.201958,-0.194539,0.561351,-0.507688,-0.053596,0.638232,...,-0.948391,1.039589,0.716911,-0.356180,0.255539,0.017466,-1.294748,1.258572,0.918162,-0.158343,0.867902,0.369649,0.582919,-0.297028,-0.075817,0.205256,-1.499854,-0.275486,-0.125899,0.307774,0.851638,-0.356368,0.687932,0.799502,0.421377,-0.190017,0.142072,0.495524,-0.906195,-0.577844,0.274673,-0.498631,-0.242185,-1.168809,-1.281136,0.845309,-0.071417,-0.117239,-0.254906,0.139219
3,0.274926,-0.126301,-0.631955,-0.286804,0.469159,-0.762826,0.159990,1.675737,-0.120078,0.110046,-0.167605,1.583477,-0.271240,0.077145,0.680479,-0.507890,-0.311672,-1.221155,-0.383460,0.475521,-0.459136,0.312512,-0.620329,0.632345,0.208755,0.124423,-0.471523,-0.183993,-0.269986,1.444585,0.716014,-0.411494,0.173376,0.329115,0.033095,-0.483648,0.599815,-0.698767,-0.203311,-0.586017,...,-0.162865,0.416873,-0.022237,0.676482,-0.455157,-0.572454,-0.305168,0.015174,0.532305,0.258627,-0.220967,-0.498072,0.780990,-0.697550,-0.321884,0.224319,-0.536807,0.077742,-0.061872,-0.500547,-0.096822,0.386803,0.850100,0.064806,0.855331,-0.766604,0.099282,-1.130597,-0.268207,0.918061,0.898908,0.867799,-0.327585,0.077369,-0.608452,0.575811,0.307833,-1.364767,-1.187833,-0.072180
4,0.495336,-0.093080,0.436682,0.481115,0.724366,0.243258,-0.056466,0.695377,0.736706,0.901954,-0.706844,0.411791,0.627986,-2.121240,0.315191,0.541978,-0.120152,-0.442829,0.794711,1.103678,-1.249040,1.409223,-0.688706,0.893563,1.034499,-0.466004,-0.136550,-0.450975,0.555152,0.800661,0.027372,-1.081319,0.688680,0.412585,-0.397247,0.022942,0.856043,-0.757594,0.390109,1.318436,...,0.412409,0.211424,-0.133326,-1.269488,-0.868590,0.015597,-0.391756,0.108011,-0.694229,-0.946797,0.305675,-0.469952,0.967952,-0.496815,0.318360,-0.939234,-0.834144,0.697586,-0.165346,1.211780,0.402460,-0.225748,-0.300347,0.902606,0.714769,-1.142596,0.572049,-1.560118,-1.621383,0.660234,1.247

# 3. Regressão Logística

In [ ]:
class LosgisticRegression():
    def __init__(self, alpha=0.001, epochs=100):
        self.alpha = alpha 
        self.epochs = epochs
        self.errors = []
        

    def sigmoide(self, x, w):
        return 1/(1+np.exp(-x @ w))

    def loss_cross_entropy(self, sum_error):
        return (1/(2*self.N))*sum_error

    # def gradient_descendent(self):
    #     return True
    
    # def stocatic_gradiend_descendent(self):
    #     return True

    def fit(self, x_train, y_train):
        self.D = x_train.shape[1]
        self.N = x_train.shape[0]
        self.w = np.zeros(self.D + 1)
        x_train = np.hstack((np.ones((self.N, 1)), x_train))

        for epoch in range(self.epochs):
            sum_error, sum_ = 0, 0
            for i in range(self.N):
                current_error = y_train[i] - self.sigmoide(x_train[i], self.w)
                sum_error += current_error**2
                sum_ += current_error * np.transpose(x_train[i])
            self.w = np.transpose(self.w) + (self.alpha * (1/self.N) * sum_)
            self.errors.append(self.loss_cross_entropy(sum_error))

    def predict(self, x_test):
        x_test = np.hstack((np.ones((x_test.shape[0], 1)), x_test))
        return [1 if self.sigmoide(row, self.w) >= 0.5 else 0 for row in x_test]
        

## Treino

In [ ]:
num_epochs = 250
alphas = [0.01, 0.001, 0.0001, 0.00001, 5, 3, 10]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_validation, Y_train, Y_validation = train_test_split(x_train, y_train["classification"], test_size=0.2, random_state=42)
len(X_train), len(X_validation), len(Y_train), len(Y_validation)

(884, 222, 884, 222)

In [ ]:
print("------ GRID SEARCH ---------")
accs = []
for alpha in alphas:
    lg = LosgisticRegression(alpha, num_epochs)
    lg.fit(X_train, Y_train.values)
    y_predict = lg.predict(X_validation)
    print(" ------------ ALPHA = " + str(alpha) + " --------------------")
    print('Test Accuraccy: ', accuracy_score(Y_validation, y_predict))
    print('\nConfusion Matrix:')
    print(confusion_matrix(Y_validation, y_predict))
    print(classification_report(Y_validation, y_predict, output_dict=True))
    accs.append((alpha, accuracy_score(Y_validation, y_predict)))
    report = classification_report(Y_validation, y_predict, output_dict=True)



------ GRID SEARCH ---------
 ------------ ALPHA = 0.01 --------------------
Test Accuraccy:  0.8063063063063063

Confusion Matrix:
[[104   7]
 [ 36  75]]
{'0': {'precision': 0.7428571428571429, 'recall': 0.9369369369369369, 'f1-score': 0.8286852589641435, 'support': 111}, '1': {'precision': 0.9146341463414634, 'recall': 0.6756756756756757, 'f1-score': 0.7772020725388601, 'support': 111}, 'accuracy': 0.8063063063063063, 'macro avg': {'precision': 0.8287456445993031, 'recall': 0.8063063063063063, 'f1-score': 0.8029436657515018, 'support': 222}, 'weighted avg': {'precision': 0.8287456445993031, 'recall': 0.8063063063063063, 'f1-score': 0.8029436657515018, 'support': 222}}
 ------------ ALPHA = 0.001 --------------------
Test Accuraccy:  0.6531531531531531

Confusion Matrix:
[[110   1]
 [ 76  35]]
{'0': {'precision': 0.5913978494623656, 'recall': 0.990990990990991, 'f1-score': 0.7407407407407408, 'support': 111}, '1': {'precision': 0.9722222222222222, 'recall': 0.3153153153153153, 'f1-sco

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in exp
  if __name__ == '__main__':


 ------------ ALPHA = 10 --------------------
Test Accuraccy:  0.8153153153153153

Confusion Matrix:
[[105   6]
 [ 35  76]]
{'0': {'precision': 0.75, 'recall': 0.9459459459459459, 'f1-score': 0.8366533864541832, 'support': 111}, '1': {'precision': 0.926829268292683, 'recall': 0.6846846846846847, 'f1-score': 0.7875647668393783, 'support': 111}, 'accuracy': 0.8153153153153153, 'macro avg': {'precision': 0.8384146341463414, 'recall': 0.8153153153153153, 'f1-score': 0.8121090766467807, 'support': 222}, 'weighted avg': {'precision': 0.8384146341463414, 'recall': 0.8153153153153153, 'f1-score': 0.8121090766467807, 'support': 222}}


In [ ]:
lista_ = [acc for alpha, acc in accs]
best_alpha = accs[lista_.index(max(lista_))][0]
best_alpha

5

In [ ]:
lg = LosgisticRegression(best_alpha, num_epochs)
lg.fit(x_train, y_train['classification'].values)

## Predição

In [ ]:
y_predict = lg.predict(x_test)

## Métricas

In [ ]:
print('Test Accuraccy: ', accuracy_score(y_test, y_predict))
print('\nConfusion Matrix:')
print(confusion_matrix(y_test, y_predict))
report = classification_report(y_test, y_predict, output_dict=True)

Test Accuraccy:  0.8375451263537906

Confusion Matrix:
[[124   6]
 [ 39 108]]


In [ ]:
report

{'0': {'f1-score': 0.8464163822525597,
  'precision': 0.7607361963190185,
  'recall': 0.9538461538461539,
  'support': 130},
 '1': {'f1-score': 0.8275862068965517,
  'precision': 0.9473684210526315,
  'recall': 0.7346938775510204,
  'support': 147},
 'accuracy': 0.8375451263537906,
 'macro avg': {'f1-score': 0.8370012945745557,
  'precision': 0.8540523086858249,
  'recall': 0.8442700156985872,
  'support': 277},
 'weighted avg': {'f1-score': 0.8364234733091186,
  'precision': 0.8597792903112246,
  'recall': 0.8375451263537906,
  'support': 277}}

## Salvando resultados

In [ ]:
resultados = pd.DataFrame({
    'precision': report['weighted avg']['precision'], 
    'recall': report['weighted avg']['recall'], 
    'f1score': report['weighted avg']['f1-score'], 
    'y_predict': [y_predict],
    'y_real': [y_test['classification'].values],
    'hiperparams': str({'alpha': best_alpha})
})
resultados.to_json('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/05. Resultados/5.1. Resultados dos modelos/1. Regressão Logística/regressao_logistica.json')

In [ ]:
pd.read_json('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/05. Resultados/5.1. Resultados dos modelos/1. Regressão Logística/regressao_logistica.json')

,precision,recall,f1score,y_predict,y_real,hiperparams
0,0.859779,0.837545,0.836423,"[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, ...","[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, ...",{'alpha': 5}


# 4. Análise Discriminante Gaussiano

In [ ]:
import numpy as np
from scipy.stats import multivariate_normal

class GaussianDiscriminant():
  def __init__(self):
    pass

  def fitAGD(self, X, y):
    self.__classes = np.unique(y) # pegando as classes e ocorrencias
    numClasses = len(self.__classes) #numero de classes
    n, numFeatures = X.shape # numero de linhas e colunas do dataset 
    
    self.__sigma = 0
    self.__probabilidadeClasses = np.zeros((numClasses, 1))
    self.__media = np.zeros((numClasses, numFeatures)) # criando a lista da media das classes por feature

    for i in range(numClasses):
        # flatnonzero retorna os indices em que os valores são diferentes de zero. O retorno é uma lista flat. 
        # A comparação y == classes vai retornar um array de 0's e 1's. 
        # E portanto a operaçao final vai pegar todos os indices em que a classe é igual ao y.
        indices = np.flatnonzero(y == self.__classes[i])

        self.__probabilidadeClasses[i] = len(indices)/ n
        self.__media[i] = np.mean(X[indices], axis=0)
        self.__sigma += np.cov(X[indices].T) *(len(indices)-1)
        
    self.__sigma /= n
    
    return {'media': self.__media, 'covar': self.__sigma, 'classes': self.__classes, 'numRows': n, 'numClasses': numClasses, 
            'numFeatures': numFeatures, 'probabilidadeClasses': self.__probabilidadeClasses }

  def predict(self,X):
    # Usando a distribuição gaussiana multivariada da biblioteca scipy.stats
    # Basicamente o calculo da função densidade de probabilidade visto na imagem abaixo.
    pdf = lambda mean: multivariate_normal.pdf(X, mean=mean, cov=self.__sigma)

    # Apply a function to 1-D slices along the given axis
    # No caso aplica a função fdp no axis 1 das médias
    y_probs = np.apply_along_axis(pdf, 1, self.__media) * self.__probabilidadeClasses

    return self.__classes[np.argmax(y_probs, axis=0)]


## Treino

In [ ]:
agd = GaussianDiscriminant()
fit = agd.fitAGD(x_train.to_numpy(), y_train)


## Predição

In [ ]:
y_predict = agd.predict(x_test.to_numpy())

## Métricas

In [ ]:
print('Test Accuraccy: ', accuracy_score(y_test, y_predict))
print('\nConfusion Matrix:')
print(confusion_matrix(y_test, y_predict))
report = classification_report(y_test, y_predict, output_dict=True)

Test Accuraccy:  0.855595667870036

Confusion Matrix:
[[117  13]
 [ 27 120]]


In [ ]:
report

{'0': {'f1-score': 0.854014598540146,
  'precision': 0.8125,
  'recall': 0.9,
  'support': 130},
 '1': {'f1-score': 0.8571428571428571,
  'precision': 0.9022556390977443,
  'recall': 0.8163265306122449,
  'support': 147},
 'accuracy': 0.855595667870036,
 'macro avg': {'f1-score': 0.8555787278415016,
  'precision': 0.8573778195488722,
  'recall': 0.8581632653061224,
  'support': 277},
 'weighted avg': {'f1-score': 0.8556747213365307,
  'precision': 0.8601320539616188,
  'recall': 0.855595667870036,
  'support': 277}}

## Salvando resultados

In [ ]:
resultados = pd.DataFrame({
    'precision': report['weighted avg']['precision'], 
    'recall': report['weighted avg']['recall'], 
    'f1score': report['weighted avg']['f1-score'], 
    'y_predict': [y_predict],
    'y_real': [y_test['classification'].values],
})
resultados.to_json('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/05. Resultados/5.1. Resultados dos modelos/3. Análise Discriminante Gaussiano/agd.json')

In [ ]:
resultados

,precision,recall,f1score,y_predict,y_real
0,0.860132,0.855596,0.855675,"[1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, ...","[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, ..."


In [ ]:
pd.read_json('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/05. Resultados/5.1. Resultados dos modelos/3. Análise Discriminante Gaussiano/agd.json')

,precision,recall,f1score,y_predict,y_real
0,0.860132,0.855596,0.855675,"[1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, ...","[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, ..."


# 5. Árvore de Decisão

## Treino

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

max_depths = [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]
params = {'criterion': ['gini','entropy'], 'max_depth': max_depths}

clf = GridSearchCV(DecisionTreeClassifier(), params, verbose=2)
    
print("[Árvore de Decisão] Treinando modelos...")
clf.fit(x_train, y_train['classification'].values)

[Árvore de Decisão] Treinando modelos...
Fitting 5 folds for each of 26 candidates, totalling 130 fits
[CV] criterion=gini, max_depth=1 .....................................
[CV] ...................... criterion=gini, max_depth=1, total=   0.0s
[CV] criterion=gini, max_depth=1 .....................................
[CV] ...................... criterion=gini, max_depth=1, total=   0.0s
[CV] criterion=gini, max_depth=1 .....................................
[CV] ...................... criterion=gini, max_depth=1, total=   0.0s
[CV] criterion=gini, max_depth=1 .....................................
[CV] ...................... criterion=gini, max_depth=1, total=   0.0s
[CV] criterion=gini, max_depth=1 .....................................
[CV] ...................... criterion=gini, max_depth=1, total=   0.0s
[CV] criterion=gini, max_depth=5 .....................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ...................... criterion=gini, max_depth=5, total=   0.1s
[CV] criterion=gini, max_depth=5 .....................................
[CV] ...................... criterion=gini, max_depth=5, total=   0.1s
[CV] criterion=gini, max_depth=5 .....................................
[CV] ...................... criterion=gini, max_depth=5, total=   0.1s
[CV] criterion=gini, max_depth=5 .....................................
[CV] ...................... criterion=gini, max_depth=5, total=   0.1s
[CV] criterion=gini, max_depth=5 .....................................
[CV] ...................... criterion=gini, max_depth=5, total=   0.1s
[CV] criterion=gini, max_depth=10 ....................................
[CV] ..................... criterion=gini, max_depth=10, total=   0.2s
[CV] criterion=gini, max_depth=10 ....................................
[CV] ..................... criterion=gini, max_depth=10, total=   0.2s
[CV] criterion=gini, max_depth=10 ....................................
[CV] .

[Parallel(n_jobs=1)]: Done 130 out of 130 | elapsed:   33.7s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'criterion': ['gini', 'entropy'],
                

In [ ]:
params = clf.best_params_
print("Parâmetros escolhidos para Árvore de Decisão: ", clf.best_params_)

Parâmetros escolhidos para Árvore de Decisão:  {'criterion': 'entropy', 'max_depth': 40}


## Predição

In [ ]:
y_predict = clf.predict(x_test)

## Métricas

In [ ]:
print('Test Accuraccy: ', accuracy_score(y_test, y_predict))
print('\nConfusion Matrix:')
print(confusion_matrix(y_test, y_predict))
report = classification_report(y_test, y_predict, output_dict=True)
report

Test Accuraccy:  0.7292418772563177

Confusion Matrix:
[[ 92  38]
 [ 37 110]]


{'0': {'f1-score': 0.7104247104247104,
  'precision': 0.7131782945736435,
  'recall': 0.7076923076923077,
  'support': 130},
 '1': {'f1-score': 0.7457627118644067,
  'precision': 0.7432432432432432,
  'recall': 0.7482993197278912,
  'support': 147},
 'accuracy': 0.7292418772563177,
 'macro avg': {'f1-score': 0.7280937111445586,
  'precision': 0.7282107689084434,
  'recall': 0.7279958137100995,
  'support': 277},
 'weighted avg': {'f1-score': 0.7291780902501087,
  'precision': 0.7291333395354888,
  'recall': 0.7292418772563177,
  'support': 277}}

## Salvando resultados

In [ ]:
resultados = pd.DataFrame({
    'precision': report['weighted avg']['precision'], 
    'recall': report['weighted avg']['recall'], 
    'f1score': report['weighted avg']['f1-score'], 
    'y_predict': [y_predict],
    'y_real': [y_test['classification'].values],
    'hiperparams': str(clf.best_params_)
})
resultados.to_json('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/05. Resultados/5.1. Resultados dos modelos/5. Árvore de Decisão/ad.json')

In [ ]:
pd.read_json('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/05. Resultados/5.1. Resultados dos modelos/5. Árvore de Decisão/ad.json')

,precision,recall,f1score,y_predict,y_real,hiperparams
0,0.729133,0.729242,0.729178,"[0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, ...","[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, ...","{'criterion': 'entropy', 'max_depth': 40}"


# 6. Random Forest

## Treino

In [ ]:
from sklearn.ensemble import RandomForestClassifier

print("[RANDOM FOREST] Selecionando hiperparâmetros...")
num_estimators = [100, 110, 120, 130, 140, 150]
max_depths = [1, 5, 10, 15, 20, 25, 55, 60]

params = {'criterion': ['gini','entropy'],
          'n_estimators': num_estimators,
          'max_depth': max_depths
          }

# params
clt = GridSearchCV(RandomForestClassifier(), params, verbose=2)

print("[RANDOM FOREST] Treinando modelo...")
clf.fit(x_train, y_train['classification'])

[RANDOM FOREST] Selecionando hiperparâmetros...
[RANDOM FOREST] Treinando modelo...
Fitting 5 folds for each of 156 candidates, totalling 780 fits
[CV] criterion=gini, max_depth=1, n_estimators=100 ...................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .... criterion=gini, max_depth=1, n_estimators=100, total=   0.3s
[CV] criterion=gini, max_depth=1, n_estimators=100 ...................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV] .... criterion=gini, max_depth=1, n_estimators=100, total=   0.3s
[CV] criterion=gini, max_depth=1, n_estimators=100 ...................
[CV] .... criterion=gini, max_depth=1, n_estimators=100, total=   0.3s
[CV] criterion=gini, max_depth=1, n_estimators=100 ...................
[CV] .... criterion=gini, max_depth=1, n_estimators=100, total=   0.3s
[CV] criterion=gini, max_depth=1, n_estimators=100 ...................
[CV] .... criterion=gini, max_depth=1, n_estimators=100, total=   0.3s
[CV] criterion=gini, max_depth=1, n_estimators=110 ...................
[CV] .... criterion=gini, max_depth=1, n_estimators=110, total=   0.3s
[CV] criterion=gini, max_depth=1, n_estimators=110 ...................
[CV] .... criterion=gini, max_depth=1, n_estimators=110, total=   0.3s
[CV] criterion=gini, max_depth=1, n_estimators=110 ...................
[CV] .... criterion=gini, max_depth=1, n_estimators=110, total=   0.3s
[CV] criterion=gini, max_depth=1, n_estimators=110 ...................
[CV] .

[Parallel(n_jobs=1)]: Done 780 out of 780 | elapsed: 19.0min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              ra

In [ ]:
print("[RANDOM FOREST] Hiperparâmetros escolhidos para Radom Forest: ", clf.best_params_)

[RANDOM FOREST] Hiperparâmetros escolhidos para Radom Forest:  {'criterion': 'gini', 'max_depth': 45, 'n_estimators': 120}


## Predição

In [ ]:
y_predict = clf.predict(x_test)

## Métricas

In [ ]:
print('Test Accuraccy: ', accuracy_score(y_test, y_predict))
print('\nConfusion Matrix:')
print(confusion_matrix(y_test, y_predict))
report = classification_report(y_test, y_predict, output_dict=True)
report

Test Accuraccy:  0.8086642599277978

Confusion Matrix:
[[114  16]
 [ 37 110]]


{'0': {'f1-score': 0.8113879003558719,
  'precision': 0.7549668874172185,
  'recall': 0.8769230769230769,
  'support': 130},
 '1': {'f1-score': 0.8058608058608059,
  'precision': 0.873015873015873,
  'recall': 0.7482993197278912,
  'support': 147},
 'accuracy': 0.8086642599277978,
 'macro avg': {'f1-score': 0.8086243531083389,
  'precision': 0.8139913802165457,
  'recall': 0.812611198325484,
  'support': 277},
 'weighted avg': {'f1-score': 0.8084547491256383,
  'precision': 0.8176138220128945,
  'recall': 0.8086642599277978,
  'support': 277}}

## Salvando métricas

In [ ]:
resultados = pd.DataFrame({
    'precision': report['weighted avg']['precision'], 
    'recall': report['weighted avg']['recall'], 
    'f1score': report['weighted avg']['f1-score'], 
    'y_predict': [y_predict],
    'y_real': [y_test['classification'].values],
    'hiperparams': str(clf.best_params_)
})
resultados.to_json('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/05. Resultados/5.1. Resultados dos modelos/8. Random Forest/random_forest.json')

In [ ]:
pd.read_json('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/05. Resultados/5.1. Resultados dos modelos/8. Random Forest/random_forest.json')

,precision,recall,f1score,y_predict,y_real,hiperparams
0,0.817614,0.808664,0.808455,"[1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, ...","[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, ...","{'criterion': 'gini', 'max_depth': 45, 'n_esti..."
